In [ ]:
import os
from zipfile import ZipFile
from io import StringIO, BytesIO
from numpy import where
from scipy.interpolate import interp1d, interp2d
from pandas import to_numeric, read_csv
from mpcontribs.client import Client
from tqdm.notebook import tqdm
from decimal import Decimal

In [ ]:
client = Client(host="lightsources-api.materialsproject.org")

In [ ]:
name = "magmom_cobalt_copper_cerium"
project = {
    "name": name,
    "is_public": True,
    "title": "CoCuCe Magnetic Moments",
    "long_title": "Magnetic moment and coercivity for CoCuCe",
    "authors": "A.T. N`Diaye, R. Ott, A.A. Baker",
    "owner": "atndiaye@lbl.gov",
    "description": """
    Magnetic moment and coercivity are determined for CoCuCe
    through composition-dependent XAS and XMCD spectra on the Co
    L3,2 edge. Composition determined by XRF.
    """,
    "references": [
        {"label": "ALS", "url": "https://sites.google.com/a/lbl.gov/magnetism/home"}
    ],
    "other": {
        "location": "ALS Beamline 6.3.1",
        "sample": "CMSI-2-10_1 CMSI-2-10_2 CMSI-2-10_3 CMSI-2-10_4 CMSI-2-10_6"
    },
}

# client.projects.create_entry(project=project).result()
# client.projects.update_entry(pk=name, project=project).result()

In [ ]:
zip_path = "/Users/patrick/gitrepos/mp/mpcontribs-data/dataset.zip"
zip_file = ZipFile(zip_path, "r")

In [ ]:
ctable = read_csv(StringIO("""
X,		Y,		Co,		Cu,		Ce
-8.5,	37.6,	46.2,	5.3,	39.3
-8.5,	107.8,	70.0,	8.9,	15.5
-7.2,	37.9,	61.2,	4.3,	34.1
-7.2,	107.0,	88.2,	6.5,	12.3
-6.3,	37.9,	66.0,	20.2,	14.89
-6.3,	107.9,	88.7,	2.7,	9.5
-6.1,	35.4,	66.0,	20.2,	14.89
-6.1,	105.4,	88.7,	2.7,	9.5
-5.7,	34.8,	48.3,	12.5,	39.3
-5.7,	104.8,	54.9,	19.1,	15.5
-5.0,	37.1,	48.8,	8.7,	43.7
-5.0,	107.1,	64.8,	16.9,	19.2
""".replace('\t', '')))

ctable["x/y position [mm]"] = ctable["X"].astype('str') + '/' + ctable["Y"].astype('str')
ctable.attrs["name"] = "Composition Table"
ctable.attrs["meta"] = {"X": "category", "Y": "continuous"}
ctable.attrs["labels"] = {"value": "composition [%]"}
ctable.set_index("x/y position [mm]", inplace=True)
ctable.columns.name = "element"

In [ ]:
def get_concentration_functions(composition_table):

    meta = composition_table.attrs["meta"]
    elements = [col for col in composition_table.columns if col not in meta]
    x = composition_table["X"].values
    y = composition_table["Y"].values
    cats = composition_table["X"].unique()
    concentration, conc, d, y_c, functions = {}, {}, {}, {}, {}

    for el in elements:
        concentration[el] = to_numeric(composition_table[el].values) / 100.0
        conc[el], d[el], y_c[el] = {}, {}, {}

        if meta["X"] == "category":
            for i in cats:
                k = "{:06.2f}".format(float(i))
                y_c[el][k] = to_numeric(y[where(x == i)])
                conc[el][k] = to_numeric(concentration[el][where(x == i)])
                d[el][k] = interp1d(y_c[el][k], conc[el][k])

            functions[el] = lambda a, b, el=el: d[el][a](b)

        else:
            functions[el] = interp2d(float(x), float(y), concentration[el])

    return functions

conc_funcs = get_concentration_functions(ctable)
del ctable["X"]
del ctable["Y"]
ctable

In [ ]:
contributions = []

for info in tqdm(zip_file.infolist()):
    # positions.x/y from filename, <scan-id>_<meas-element>_<X>_<Y>.csv
    fn = os.path.splitext(info.filename)[0]
    element, x, y = fn.rsplit("_", 4)
    identifier = f"{element}@{x}/{y}"
    csv = zip_file.read(info.filename)

    df = read_csv(BytesIO(csv))
    df = df[["Energy", "XAS", "XMCD"]]
    df.set_index("Energy", inplace=True)
    df.index.name = "Energy [eV]"
    df.columns.name = "spectral type"
    df.attrs["name"] = f"{element}-XAS/XMCD"
    df.attrs["title"] = f"XAS and XMCD Spectra for {element}"
    df.attrs["labels"] = {"value": "a.u."}

    contrib = {"project": name, "identifier": identifier, "is_public": True, "data": {}}
    contrib["data"]["position"] = {k: f"{v} mm" for k, v in zip(["x", "y"], [x, y])}
    contrib["data"]["composition"] = {}
        
    for el, f in conc_funcs.items():
        try:
            contrib["data"]["composition"][el] = f"{f(x, y) * 100.} %"
        except KeyError:
            continue
            
    contrib["formula"] = "".join([
        "{}{}".format(el, int(round(Decimal(comp.split()[0]))))
        for el, comp in contrib["data"]["composition"].items()
    ])

    for y in ["XAS", "XMCD"]:
        contrib["data"][y] = {"min": df[y].min(), "max": df[y].max()}
    
    contrib["tables"] = [ctable, df]
    contributions.append(contrib)
    
len(contributions)

In [ ]:
client.delete_contributions(name)
client.submit_contributions(contributions, ignore_dupes=True)